# Collaborative Filtering: Hiperparametrización

## Importación de librerías

In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)

2.3.0+cu121
12.1


In [2]:
# Reinstala PyTorch 2.3.0 con CUDA 12.1 (estable en Colab)
!pip install torch==2.3.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Instala PyG y extensiones con soporte oficial
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv \
  -f https://data.pyg.org/whl/torch-2.3.0+cu121.html
!pip install -q torch-geometric

Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.5 MB/s eta 0:00:00


In [3]:
!pip install -q git+https://github.com/snap-stanford/deepsnap.git
!pip install -U -q PyDrive

  Preparing metadata (setup.py) ... done


In [4]:
import torch
import random
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from torch import nn, optim, Tensor
from torch_sparse import SparseTensor, matmul

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing


## Carga de datos

### Definición de funciones

Comenzamos definiendo las funciones necesarias para construir los nodos y las aristas del grafo a partir de los datos de usuarios, libros e interacciones.

In [5]:
# Cargar nodos de usuarios y libros
def load_node_csv(path, index_col):
    """
    Carga un CSV que contiene información sobre los nodos.

    Args:
        path (str): Ruta al archivo CSV.
        index_col (str): Nombre de la columna que se usará como índice.

    Returns:
        dict: Diccionario que mapea los valores del índice del CSV a IDs de nodo.
    """
    df = pd.read_csv(path, index_col=index_col)
    mapping = {index: i for i, index in enumerate(df.index.unique())}
    return mapping

# Cargar aristas entre usuarios y libros
def load_edge_csv(df, src_index_col, src_mapping, dst_index_col, dst_mapping, link_index_col, rating_threshold=4):
    """
    Carga aristas entre usuarios e ítems (por ejemplo, libros) a partir de un DataFrame.

    Args:
        df (DataFrame): DataFrame que contiene las interacciones usuario-ítem.
        src_index_col (str): Nombre de la columna con los IDs de usuario.
        src_mapping (dict): Diccionario que mapea los IDs de usuario a índices de nodo.
        dst_index_col (str): Nombre de la columna con los IDs de ítems.
        dst_mapping (dict): Diccionario que mapea los IDs de ítems a índices de nodo.
        link_index_col (str): Nombre de la columna que contiene la interacción (por ejemplo, rating).
        rating_threshold (int, opcional): Umbral mínimo para considerar una interacción como positiva. Por defecto es 4.

    Returns:
        torch.Tensor: Matriz 2xN que contiene los pares de nodos conectados por N aristas.
    """
    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_attr = torch.from_numpy(df[link_index_col].values).view(-1, 1).to(torch.long) >= rating_threshold

    edge_index = [[], []]
    for i in range(edge_attr.shape[0]):
        if edge_attr[i]:
            edge_index[0].append(src[i])
            edge_index[1].append(dst[i])

    return torch.tensor(edge_index)

Implementamos una función que realiza muestreo estructurado negativo para generar mini-batches compuestos por usuarios, ítems positivos e ítems negativos, necesarios para entrenar el modelo basado en grafos.

In [6]:
# Función que selecciona aleatoriamente un mini-lote de muestras positivas y negativas
def sample_mini_batch(batch_size, edge_index):
    """
    Selecciona aleatoriamente los índices de un mini-lote dado un grafo de adyacencia.

    Args:
        batch_size (int): tamaño del mini-lote
        edge_index (torch.Tensor): matriz 2 x N con los bordes del grafo

    Returns:
        tuple: índices de usuarios, índices de ítems positivos, índices de ítems negativos
    """
    edges = structured_negative_sampling(edge_index)
    edges = torch.stack(edges, dim=0)
    indices = random.choices([i for i in range(edges[0].shape[0])], k=batch_size)
    batch = edges[:, indices]
    user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
    return user_indices, pos_item_indices, neg_item_indices

### Cargamos y filtramos los datos

In [7]:
user_id_path = '/content/user_id_map.csv'
book_id_path = '/content/book_id_map.csv'
interactions_path = '/content/interactions_filtered.csv'
books_path = '/content/books_authors_genres.csv'

In [8]:
interactions = pd.read_csv(interactions_path)
books = pd.read_csv(books_path)

Aplicamos un filtrado para eliminar ítems con pocas interacciones y usuarios poco activos, mejorando así la calidad del grafo de interacciones. Esto reduce la dispersión y mejora el aprendizaje del modelo.

In [9]:
item_inter_counts = interactions.groupby("book_id").size()
popular_books = item_inter_counts[item_inter_counts >= 10].index
interactions = interactions[interactions["book_id"].isin(popular_books)]

user_inter_counts = interactions.groupby('user_id').size()
active_users = user_inter_counts[user_inter_counts >= 4].index
interactions = interactions[interactions['user_id'].isin(active_users)]

print(f"Interactions filtrado: {interactions.shape}")

num_users = interactions['user_id'].nunique()
num_items = interactions['book_id'].nunique()
print(f"{num_users=}, {num_items=}")

Interactions filtrado: (16803, 5)
num_users=3310, num_items=947


Aplicamos las funciones para crear los nodos y las aristas.

In [10]:
user_mapping = load_node_csv(user_id_path, index_col='user_id_csv')
book_mapping = load_node_csv(book_id_path, index_col='book_id_csv')

In [11]:
edge_index = load_edge_csv(
    interactions,
    src_index_col='user_id',
    src_mapping=user_mapping,
    dst_index_col='book_id',
    dst_mapping=book_mapping,
    link_index_col='rating',
    rating_threshold=4
)

# Verifica la forma de edge_index
print("edge_index:", edge_index)
print("edge_index.shape:", edge_index.shape)

edge_index: tensor([[    14,     14,     14,  ..., 873215, 873215, 873215],
        [ 11831,  10943,  10570,  ...,  39638,  39637,  39640]])
edge_index.shape: torch.Size([2, 16803])


Dividimos las interacciones del grafo entre usuarios y libros en tres subconjuntos: entrenamiento, validación y prueba, siguiendo una proporción del 80%, 10% y 10% respectivamente.

In [12]:
# Dividir los bordes del grafo en un 80/10/10 para entrenamiento, validación y prueba
num_users, num_books = len(user_mapping), len(book_mapping)
num_interacciones = edge_index.shape[1]
todos_los_indices = [i for i in range(num_interacciones)]

# 80% entrenamiento, 10% validación y 10% prueba
train_indices, test_indices = train_test_split(todos_los_indices, test_size=0.2, random_state=1)
val_indices, test_indices = train_test_split(test_indices, test_size=0.5, random_state=1)

train_edge_index = edge_index[:, train_indices]
val_edge_index = edge_index[:, val_indices]
test_edge_index = edge_index[:, test_indices]

In [13]:
print("train_edge_index.shape:", train_edge_index.shape)
print("val_edge_index.shape:", val_edge_index.shape)
print("test_edge_index.shape:", test_edge_index.shape)

train_edge_index.shape: torch.Size([2, 13442])
val_edge_index.shape: torch.Size([2, 1680])
test_edge_index.shape: torch.Size([2, 1681])


Convertimos los índices de los bordes del grafo en tensores dispersos (SparseTensor) para optimizar el almacenamiento y las operaciones sobre grafos grandes.

In [14]:
train_sparse_edge_index = SparseTensor(
    row=train_edge_index[0],
    col=train_edge_index[1],
    sparse_sizes=(num_users + num_books, num_users + num_books)
)

val_sparse_edge_index = SparseTensor(
    row=val_edge_index[0],
    col=val_edge_index[1],
    sparse_sizes=(num_users + num_books, num_users + num_books)
)

test_sparse_edge_index = SparseTensor(
    row=test_edge_index[0],
    col=test_edge_index[1],
    sparse_sizes=(num_users + num_books, num_users + num_books)
)

## Implementación de LightGCN

### Definición del modelo

A continuación, definimos el modelo LightGCN (Light Graph Convolutional Network), una red neuronal propuesta en el artículo LightGCN: Simplifying and Powering Graph Convolution Network for Recommendation.

Este modelo está diseñado específicamente para sistemas de recomendación, eliminando componentes innecesarios como funciones de activación o capas de proyección, y enfocándose únicamente en la agregación de embeddings a lo largo de un grafo bipartito de usuarios e ítems.

La idea principal es realizar una propagación de mensajes ligera a través del grafo para capturar la estructura de las interacciones.
El modelo inicializa embeddings para usuarios e ítems, y luego los actualiza mediante varias capas de propagación sobre la matriz de adyacencia normalizada. Finalmente, se promedia la información agregada en cada capa para obtener una representación final que captura diferentes niveles de conectividad.

In [15]:
# Definimos el modelo LightGCN
class LightGCN(MessagePassing):
    """
    Modelo LightGCN propuesto en https://arxiv.org/abs/2002.02126
    """

    def __init__(self, num_users, num_items, embedding_dim=16, K=2, add_self_loops=False):
        """
        Inicializa el modelo LightGCN.

        Args:
            num_users (int): Número de usuarios
            num_items (int): Número de ítems (libros)
            embedding_dim (int, opcional): Dimensión de los embeddings. Por defecto 16.
            K (int, opcional): Número de capas de propagación de mensajes. Por defecto 2.
            add_self_loops (bool, opcional): Si se añaden bucles propios en la propagación. Por defecto False.
        """
        super().__init__()
        self.num_users, self.num_items = num_users, num_items
        self.embedding_dim, self.K = embedding_dim, K
        self.add_self_loops = add_self_loops

        # Embeddings iniciales de usuarios e ítems: e_u^0 y e_i^0
        self.users_emb = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.embedding_dim)
        self.items_emb = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.embedding_dim)

        # Inicializamos los embeddings de usuarios e ítems con una distribución normal
        nn.init.normal_(self.users_emb.weight, std=0.1)
        nn.init.normal_(self.items_emb.weight, std=0.1)

    def forward(self, edge_index: SparseTensor):
        """
        Propagación hacia adelante del modelo LightGCN.

        Args:
            edge_index (SparseTensor): Matriz de adyacencia

        Returns:
            tuple (Tensor): Embeddings finales y originales de usuarios e ítems (e_u^K, e_u^0, e_i^K, e_i^0)
        """
        # Calculamos \tilde{A}: matriz de adyacencia normalizada simétricamente
        edge_index_norm = gcn_norm(edge_index, add_self_loops=self.add_self_loops)

        # Embedding inicial conjunto E^0 (usuarios + ítems)
        emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight])
        embs = [emb_0]
        emb_k = emb_0

        # Propagación por K capas
        for i in range(self.K):
            emb_k = self.propagate(edge_index_norm, x=emb_k)
            embs.append(emb_k)

        # Promedio de los embeddings generados en cada capa (E^K)
        embs = torch.stack(embs, dim=1)
        emb_final = torch.mean(embs, dim=1)

        # Separamos e_u^K y e_i^K (representaciones finales de usuarios e ítems)
        users_emb_final, items_emb_final = torch.split(emb_final, [self.num_users, self.num_items])

        # Devolvemos embeddings finales y los embeddings originales (iniciales)
        return users_emb_final, self.users_emb.weight, items_emb_final, self.items_emb.weight

    def message(self, x_j: Tensor) -> Tensor:
        """Devuelve los mensajes recibidos de los nodos vecinos."""
        return x_j

    def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
        """Multiplica la matriz de adyacencia normalizada con los embeddings."""
        return matmul(adj_t, x)

# Instanciamos el modelo
model = LightGCN(num_users, num_books)

### Función de pérdida

Utilizamos la pérdida `Bayesian Personalized Ranking (BPR)`, una función de tipo comparativa por pares que favorece que, para cada usuario, las predicciones asociadas a ejemplos positivos sean más altas que las de los ejemplos negativos.

In [16]:
def bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_val):
    """
    Función de pérdida Bayesian Personalized Ranking, descrita en https://arxiv.org/abs/1205.2618

    Args:
        users_emb_final (torch.Tensor): e_u_k
        users_emb_0 (torch.Tensor): e_u_0
        pos_items_emb_final (torch.Tensor): e_i_k de los ítems positivos
        pos_items_emb_0 (torch.Tensor): e_i_0 de los ítems positivos
        neg_items_emb_final (torch.Tensor): e_i_k de los ítems negativos
        neg_items_emb_0 (torch.Tensor): e_i_0 de los ítems negativos
        lambda_val (float): valor lambda para el término de regularización

    Returns:
        torch.Tensor: valor escalar de la pérdida BPR
    """
    reg_loss = lambda_val * (users_emb_0.norm(2).pow(2) +
                             pos_items_emb_0.norm(2).pow(2) +
                             neg_items_emb_0.norm(2).pow(2))  # Pérdida L2

    pos_scores = torch.mul(users_emb_final, pos_items_emb_final)
    pos_scores = torch.sum(pos_scores, dim=-1)  # Puntuaciones predichas de los ejemplos positivos
    neg_scores = torch.mul(users_emb_final, neg_items_emb_final)
    neg_scores = torch.sum(neg_scores, dim=-1)  # Puntuaciones predichas de los ejemplos negativos

    loss = -torch.mean(torch.nn.functional.softplus(pos_scores - neg_scores)) + reg_loss

    return loss

### Métricas de evaluación

Para evaluar la el sistema de recomendación, empleamos métricas estándar centradas en las primeras posiciones del ranking generado para cada usuario. Concretamente, utilizamos `precisión@20`, `recall@20` y `nDCG@20`, que nos permiten medir tanto la relevancia como la ordenación de los ítems recomendados. Estas métricas se enfocan en los 20 primeros elementos recomendados.

In [17]:
# Función auxiliar para obtener N_u (ítems positivos por usuario)
def get_user_positive_items(edge_index):
    """
    Genera un diccionario con los ítems positivos para cada usuario

    Args:
        edge_index (torch.Tensor): lista de aristas de tamaño 2 por N

    Returns:
        dict: diccionario con los ítems positivos de cada usuario
    """
    user_pos_items = {}
    for i in range(edge_index.shape[1]):
        user = edge_index[0][i].item()
        item = edge_index[1][i].item()
        if user not in user_pos_items:
            user_pos_items[user] = []
        user_pos_items[user].append(item)
    return user_pos_items

In [18]:
# Calcula recall@K y precision@K
def RecallPrecision_ATk(groundTruth, r, k):
    """
    Calcula recall @ k y precision @ k

    Args:
        groundTruth (list): lista de listas con los ítems altamente valorados por cada usuario
        r (list): lista de listas que indica si cada ítem recomendado en top k para cada usuario
            es un ítem relevante en el top k ground truth o no
        k (int): determina los primeros k ítems para calcular precision y recall

    Returns:
        tuple: recall @ k, precision @ k
    """
    num_correct_pred = torch.sum(r, dim=-1)  # Número de ítems correctamente predichos por usuario
    # Número de ítems que cada usuario valoró positivamente en el conjunto de prueba
    user_num_liked = torch.Tensor([len(groundTruth[i])
                                  for i in range(len(groundTruth))])
    recall = torch.mean(num_correct_pred / user_num_liked)
    precision = torch.mean(num_correct_pred) / k
    return recall.item(), precision.item()

In [19]:
# Calcula NDCG@K
def NDCGatK_r(groundTruth, r, k):
    """
    Calcula Normalized Discounted Cumulative Gain (NDCG) @ k

    Args:
        groundTruth (list): lista de listas con los ítems altamente valorados por cada usuario
        r (list): lista de listas que indica si cada ítem recomendado en top k para cada usuario
            es un ítem relevante en el top k ground truth o no
        k (int): determina los primeros k ítems para calcular ndcg

    Returns:
        float: ndcg @ k
    """
    assert len(r) == len(groundTruth)

    test_matrix = torch.zeros((len(r), k))

    for i, items in enumerate(groundTruth):
        length = min(len(items), k)
        test_matrix[i, :length] = 1
    max_r = test_matrix
    idcg = torch.sum(max_r * 1. / torch.log2(torch.arange(2, k + 2)), axis=1)
    dcg = r * (1. / torch.log2(torch.arange(2, k + 2)))
    dcg = torch.sum(dcg, axis=1)
    idcg[idcg == 0.] = 1.
    ndcg = dcg / idcg
    ndcg[torch.isnan(ndcg)] = 0.
    return torch.mean(ndcg).item()

Definimos la función que evalúa en batches las métricas recall, precisión y NDCG para evitar problemas de memoria, procesando recomendaciones para un conjunto de usuarios.

In [20]:
def get_metrics(model, edge_index, exclude_edge_indices, k, batch_size=512):
    """
    Calcula las métricas de evaluación en batches para evitar desbordamiento de memoria.

    Esta función evalúa el desempeño del modelo LightGCN calculando las métricas
    de recall, precisión y NDCG en el top-k ítems recomendados para cada usuario.
    Para manejar grandes conjuntos de datos, realiza el cálculo en lotes (batches)
    de usuarios, excluyendo ítems ya vistos en entrenamiento para evitar sesgos en la evaluación.

    Args:
        model (LightGCN): modelo LightGCN entrenado
        edge_index (torch.Tensor): lista 2 x N de aristas para el conjunto a evaluar
        exclude_edge_indices (list[torch.Tensor]): lista de aristas a excluir del ranking (por ejemplo, entrenamiento)
        k (int): número de ítems top-k para evaluar
        batch_size (int): número de usuarios por lote

    Returns:
        tuple: recall @ k, precisión @ k, ndcg @ k
    """
    user_embedding = model.users_emb.weight.detach()
    item_embedding = model.items_emb.weight.detach()

    num_users = user_embedding.shape[0]
    users = edge_index[0].unique()

    test_user_pos_items = get_user_positive_items(edge_index)
    test_user_pos_items_list = [test_user_pos_items[user.item()] for user in users]

    recall_list = []
    precision_list = []
    ndcg_list = []

    for start in range(0, len(users), batch_size):
        end = min(start + batch_size, len(users))
        batch_users = users[start:end]

        batch_user_emb = user_embedding[batch_users]  # (batch, emb_size)
        rating = torch.matmul(batch_user_emb, item_embedding.T)  # (batch, num_items)

        # Excluir ítems vistos en entrenamiento de la recomendación
        for exclude_edge_index in exclude_edge_indices:
            user_pos_items = get_user_positive_items(exclude_edge_index)
            for i, user in enumerate(batch_users):
                uid = user.item()
                if uid in user_pos_items:
                    rating[i, user_pos_items[uid]] = -(1 << 10)  # Excluir

        _, top_K_items = torch.topk(rating, k=k)  # (batch, k)

        r = []
        for i, user in enumerate(batch_users):
            uid = user.item()
            ground_truth_items = test_user_pos_items[uid]
            label = [item in ground_truth_items for item in top_K_items[i]]
            r.append(label)

        r = torch.tensor(np.array(r).astype('float'))

        # Calcular métricas para el batch
        batch_ground_truth = [test_user_pos_items[user.item()] for user in batch_users]
        batch_recall, batch_precision = RecallPrecision_ATk(batch_ground_truth, r, k)
        batch_ndcg = NDCGatK_r(batch_ground_truth, r, k)

        recall_list.append(batch_recall)
        precision_list.append(batch_precision)
        ndcg_list.append(batch_ndcg)

    # Promedio de métricas entre batches
    recall = sum(recall_list) / len(recall_list)
    precision = sum(precision_list) / len(precision_list)
    ndcg = sum(ndcg_list) / len(ndcg_list)

    return recall, precision, ndcg

Definimos la función que realiza la evaluación completa del modelo, calculando la pérdida BPR y las métricas recall, precisión y NDCG sobre el conjunto de evaluación.

In [21]:
def evaluation(model, edge_index, sparse_edge_index, exclude_edge_indices, k, lambda_val=1e-6):
    """
    Evalúa la pérdida del modelo y las métricas incluyendo recall, precisión y ndcg @ k

    Args:
        model (LightGCN): modelo LightGCN
        edge_index (torch.Tensor): lista 2 x N de aristas para el conjunto a evaluar
        sparse_edge_index (sparseTensor): matriz de adyacencia dispersa para el conjunto a evaluar
        exclude_edge_indices ([tipo]): lista 2 x N de aristas para excluir de la evaluación
        k (int): determina el top k de ítems para calcular las métricas
        lambda_val (float): valor lambda para la pérdida BPR

    Returns:
        tuple: pérdida BPR, recall @ k, precisión @ k, ndcg @ k
    """
    # Obtener embeddings
    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(sparse_edge_index)

    edges = structured_negative_sampling(edge_index, contains_neg_self_loops=False)

    user_indices, pos_item_indices, neg_item_indices = edges[0], edges[1], edges[2]
    users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
    pos_items_emb_final, pos_items_emb_0 = items_emb_final[pos_item_indices], items_emb_0[pos_item_indices]
    neg_items_emb_final, neg_items_emb_0 = items_emb_final[neg_item_indices], items_emb_0[neg_item_indices]

    loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_val).item()

    recall, precision, ndcg = get_metrics(model, edge_index, exclude_edge_indices, k, batch_size = 128)

    return loss, recall, precision, ndcg

## Entrenamiento

Definimos las variables para los parámetros clave para el entrenamiento y evaluación del modelo.

In [ ]:
ITERATIONS = 1000
ITERS_PER_EVAL = 200
ITERS_PER_LR_DECAY = 200
PATIENCE = 50
K = 20
LAMBDA = 1e-6
NUM_USERS = len(user_mapping)
NUM_ITEMS = len(book_mapping)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

A continuación definimos las funciones `train_model`, `hyperparameter_search` y `create_model`. Estas funciones nos permiten crear los modelos en base a la combinación de hiperparámetros escogida y entrenarlos.

In [ ]:
def train_model(model, edge_index, train_edge_index, train_sparse_edge_index,
                val_edge_index, val_sparse_edge_index,
                iterations, batch_size, lr,
                iters_per_eval, k_eval, lambda_reg, device,
                iters_per_lr_decay=ITERS_PER_LR_DECAY,
                early_stopping_patience=PATIENCE, early_stopping_delta=1e-4):

    model = model.to(device)
    model.train()

    # Definimos el optimizador y el scheduler
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

    edge_index = edge_index.to(device)
    train_edge_index = train_edge_index.to(device)
    train_sparse_edge_index = train_sparse_edge_index.to(device)
    val_edge_index = val_edge_index.to(device)
    val_sparse_edge_index = val_sparse_edge_index.to(device)

    train_losses = []
    val_losses = []

    best_val_recall = -float('inf')
    best_model_state = None
    patience_counter = 0

    for iter in range(iterations):
        users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(train_sparse_edge_index)

        user_indices, pos_item_indices, neg_item_indices = sample_mini_batch(batch_size, train_edge_index)
        user_indices, pos_item_indices, neg_item_indices = user_indices.to(device), pos_item_indices.to(device), neg_item_indices.to(device)

        users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
        pos_items_emb_final, pos_items_emb_0 = items_emb_final[pos_item_indices], items_emb_0[pos_item_indices]
        neg_items_emb_final, neg_items_emb_0 = items_emb_final[neg_item_indices], items_emb_0[neg_item_indices]

        train_loss = bpr_loss(users_emb_final, users_emb_0,
                              pos_items_emb_final, pos_items_emb_0,
                              neg_items_emb_final, neg_items_emb_0,
                              lambda_reg)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        # Decay learning rate cada cierto número de iteraciones
        if (iter + 1) % iters_per_lr_decay == 0:
            scheduler.step()

        if (iter + 1) % iters_per_eval == 0:
            model.eval()
            val_loss, recall, precision, ndcg = evaluation(
                model, val_edge_index, val_sparse_edge_index, [train_edge_index], k_eval, lambda_reg)

            print(f"[Iteración {iter+1}/{iterations}] train_loss: {round(train_loss.item(), 5)}, "
                  f"val_loss: {round(val_loss, 5)}, val_recall@{k_eval}: {round(recall, 5)}, "
                  f"val_precision@{k_eval}: {round(precision, 5)}, val_ndcg@{k_eval}: {round(ndcg, 5)}")

            train_losses.append(train_loss.item())
            val_losses.append(val_loss)

            # Early stopping basado en recall
            if recall > best_val_recall + early_stopping_delta:
                best_val_recall = recall
                best_model_state = model.state_dict()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= early_stopping_patience:
                    print(f"Early stopping triggered at iteration {iter+1}")
                    break

            model.train()

    return train_losses, val_losses, best_val_recall, best_model_state


def hyperparameter_search(create_model, hyperparams_grid, edge_index, train_edge_index,
                          train_sparse_edge_index, val_edge_index, val_sparse_edge_index,
                          iterations, iters_per_eval,
                          k_eval, lambda_reg, device):

    # Realiza una búsqueda de hiperparámetros en una cuadrícula definida por hyperparams_grid
    keys, values = zip(*hyperparams_grid.items())
    all_combinations = list(itertools.product(*values))

    # Inicializa variables para almacenar los mejores resultados
    best_score = -float('inf')
    best_params = None
    best_model_state = None

    for v in all_combinations:
        params = dict(zip(keys, v))
        print(f"\nProbando hiperparámetros: {params}")

        model = create_model(
            num_users=NUM_USERS,
            num_items=NUM_ITEMS,
            embedding_dim=params['embedding_dim'],
            num_layers=params['num_layers']
        )

        train_losses, val_losses, val_recall, current_model_state = train_model(
            model,
            edge_index,
            train_edge_index,
            train_sparse_edge_index,
            val_edge_index,
            val_sparse_edge_index,
            iterations=iterations,
            batch_size=params['batch_size'],
            lr=params['lr'],
            iters_per_eval=iters_per_eval,
            k_eval=k_eval,
            lambda_reg=lambda_reg,
            device=device
        )

        print(f"Validación recall@{k_eval}: {val_recall}")

        # Guardar el mejor modelo basado en recall
        if val_recall > best_score:
            best_score = val_recall
            best_params = params
            best_model_state = current_model_state

    print(f"\nMejores hiperparámetros: {best_params} con recall@{k_eval}: {best_score}")
    return best_params, best_model_state

def create_model(num_users, num_items, embedding_dim, num_layers):
    return LightGCN(num_users, num_items, embedding_dim, num_layers)

In [ ]:
# Grid de hiperparámetros para búsqueda
hyperparams_grid = {
    'lr': [1e-3, 5e-4],
    'batch_size': [512, 1024],
    'embedding_dim': [16, 32],
    'num_layers': [2, 3]
}

# Ejecutar búsqueda de hiperparámetros
best_params, best_model_state = hyperparameter_search(
    create_model,
    hyperparams_grid,
    edge_index,
    train_edge_index,
    train_sparse_edge_index,
    val_edge_index,
    val_sparse_edge_index,
    iterations=ITERATIONS,
    iters_per_eval=ITERS_PER_EVAL,
    k_eval=K,
    lambda_reg=LAMBDA,
    device=device
)



Probando hiperparámetros: {'lr': 0.001, 'batch_size': 512, 'embedding_dim': 16, 'num_layers': 2}
[Iteración 200/1000] train_loss: -0.69484, val_loss: -0.69226, val_recall@20: 0.0, val_precision@20: 0.0, val_ndcg@20: 0.0
[Iteración 400/1000] train_loss: -0.6999, val_loss: -0.69369, val_recall@20: 0.07416, val_precision@20: 0.00476, val_ndcg@20: 0.06196
[Iteración 600/1000] train_loss: -0.71585, val_loss: -0.70067, val_recall@20: 0.22206, val_precision@20: 0.01401, val_ndcg@20: 0.15182
[Iteración 800/1000] train_loss: -0.75313, val_loss: -0.71838, val_recall@20: 0.3149, val_precision@20: 0.01993, val_ndcg@20: 0.19986
[Iteración 1000/1000] train_loss: -0.79847, val_loss: -0.74975, val_recall@20: 0.36418, val_precision@20: 0.02282, val_ndcg@20: 0.22588
Validación recall@20: 0.36417561026936024

Probando hiperparámetros: {'lr': 0.001, 'batch_size': 512, 'embedding_dim': 16, 'num_layers': 3}
[Iteración 200/1000] train_loss: -0.69382, val_loss: -0.69234, val_recall@20: 0.0, val_precision@20: